In [1]:
import pickle
import numpy as np

In [2]:
emb_name = 'nt_word'

In [3]:
processed_dataset_path = f"/mounts/data/proj/yihong/newhome/ConceptNetwork/eva/" + \
                         f"baseline_vectors/sentence_classification/{emb_name}"
    
train_size = 860

with open(f"{processed_dataset_path}/train/{train_size}/train.pickle", 'rb') as handle:
    train_set_to_store = pickle.load(handle)

with open(f"{processed_dataset_path}/valid/valid.pickle", 'rb') as handle:
    valid_set_to_store = pickle.load(handle)

with open(f"{processed_dataset_path}/test/test.pickle", 'rb') as handle:
    test_set_to_store = pickle.load(handle)

In [4]:
print(f"Number of languages in train: {len(train_set_to_store)}")
print(f"Number of languages in valid: {len(valid_set_to_store)}")
print(f"Number of languages in test: {len(test_set_to_store)}")

Number of languages in train: 1247
Number of languages in valid: 1247
Number of languages in test: 1247


In [5]:
# load concept networks
from gensim.models import KeyedVectors, Word2Vec

if emb_name == 'clique_word' or emb_name == 'nt_word':
    embedding_path = f"./word_embeddings_{emb_name}.kv"
    loaded_n2v = KeyedVectors.load(embedding_path)
elif emb_name == 'sentence_id':
    epochs = 50
    emb_dim = 200
    word_vec = Word2Vec.load(f"./word2vec_{epochs}_{emb_dim}.model")
    loaded_n2v = word_vec.wv
elif emb_name == 'eflomal':
    epochs = 10
    emb_dim = 200
    loaded_n2v = KeyedVectors.load(f"/mounts/data/proj/yihong/newhome/ConceptNetwork/network_related"
                                   f"/eflomal_vectors_{emb_dim}_{epochs}.wv")

In [6]:
# transform the concept (string) representations to dense vectors
mapping = {'Grace': 1, 'Violence': 2, 'Sin': 3, 'Recommendation': 4, 'Faith': 5, 'Description': 6}

In [7]:
def return_data_labels(data_dict, embedding_m, label_mapping_dict, lang):
    labels = []
    representations = []

    for pair in data_dict:

        # transform strings
        strings = pair[1]
        representation = np.zeros(embedding_m.vector_size)

        for string in strings:
            representation += np.array(embedding_m[f"{lang}:{string}"])
        representations.append(representation)

        # transform labels
        label = pair[2]
        labels.append(label_mapping_dict[label])    
    
    return representations, labels

In [8]:
def creating_train_test_set(lang, label_mapping_dict, embedding_m, train_dict, test_dict, valid_dict=None):
    representations_train, labels_train = return_data_labels(train_dict[lang], embedding_m, 
                                                             label_mapping_dict, lang)
    representations_test, labels_test = return_data_labels(test_dict[lang], embedding_m, 
                                                           label_mapping_dict, lang)
    if valid_dict is None:
        return representations_train, labels_train, representations_test, labels_test
    else:
        representations_valid, labels_valid = return_data_labels(valid_dict[lang], embedding_m, 
                                                                 label_mapping_dict, lang)
        return representations_train, labels_train, representations_valid, labels_valid, \
               representations_test, labels_test

In [9]:
# define the training language
train_language = 'arb'

r_train, l_train, r_valid, l_valid, r_test, l_test = creating_train_test_set(train_language, mapping, loaded_n2v, 
                                                                             train_set_to_store,
                                                                             test_set_to_store,
                                                                             valid_set_to_store, 
                                                                             )

In [10]:
print(f"Train data shape: {np.shape(r_train)}")
print(f"Train labels shape: {len(l_train)}")
print(f"Valid data shape: {np.shape(r_valid)}")
print(f"Valid labels shape: {len(l_valid)}")
print(f"Test data shape: {np.shape(r_test)}")
print(f"Test labels shape: {len(l_test)}")

Train data shape: (860, 200)
Train labels shape: 860
Valid data shape: (106, 200)
Valid labels shape: 106
Test data shape: (111, 200)
Test labels shape: 111


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

model_name = 'lr'
if model_name == 'lr':
    model = LogisticRegression(random_state=114514)
elif model_name == 'svm':
    model = SVC(kernel='rbf', random_state=114514)
elif model_name == 'knn':
    model = KNeighborsClassifier(n_neighbors=20)
model.fit(r_train, l_train)

/mounts/data/proj/yihong/newhome/ENTER/envs/concept-net/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


LogisticRegression(random_state=114514)

In [12]:
predictions = model.predict(r_test)

In [13]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score

print(f"Train and test on {train_language} data: ")
print(f"Accuracy score: {accuracy_score(l_test, predictions)}")
print(f"Precision score: {precision_score(l_test, predictions, average='macro')}")
print(f"Recall score: {recall_score(l_test, predictions, average='macro')}")
print(f"Macro F1 score: {f1_score(l_test, predictions, average='macro')}")

Train and test on arb data: 
Accuracy score: 0.5225225225225225
Precision score: 0.5152927120669055
Recall score: 0.48497902581387153
Macro F1 score: 0.49275937428111344


In [14]:
zero_shot_langs = list(train_set_to_store.keys())
print(f"Number of zero-shot languages: {len(zero_shot_langs) - 1}")  # filter english

f1_performance = {}

save_file_name = f"./sentence_classification/{emb_name}/{train_size}_{train_language}_{model_name}_zero_shot_results.txt"
    
with open(save_file_name, 'w', encoding='utf-8') as f:
    for test_lang in zero_shot_langs:
        if test_lang == train_language:
            continue
        _, _, r_test, l_test = creating_train_test_set(test_lang, mapping, loaded_n2v, 
                                                       train_set_to_store, test_set_to_store)
        
        if len(r_test) == 0:
            continue
            # some languages do not have any available test sets
        
        sent = f"{test_lang}:"
        print(sent)
        f.write(sent + '\n')
        
        zero_shot_predictions = model.predict(r_test)
        
        sent = f"Accuracy score: {round(accuracy_score(l_test, zero_shot_predictions), 2)}"
        print(sent)
        f.write(sent + '\n')
        
        sent = f"Precision score: {round(precision_score(l_test, zero_shot_predictions, average='macro'), 2)}"
        print(sent)
        f.write(sent + '\n')
        
        sent = f"Recall score: {round(recall_score(l_test, zero_shot_predictions, average='macro'), 2)}"
        print(sent)
        f.write(sent + '\n')
        
        sent = f"Macro F1 score: {round(f1_score(l_test, zero_shot_predictions, average='macro'), 2)}"
        print(sent)
        f.write(sent + '\n')
        
        f1_performance[test_lang] = round(f1_score(l_test, zero_shot_predictions, average='macro'), 2)
        
        sent = ''
        print(sent)
        f.write(sent + '\n')

Number of zero-shot languages: 1246
aai:
Accuracy score: 0.52
Precision score: 0.55
Recall score: 0.52
Macro F1 score: 0.51

aak:
Accuracy score: 0.54
Precision score: 0.57
Recall score: 0.55
Macro F1 score: 0.53

aau:
Accuracy score: 0.53
Precision score: 0.46
Recall score: 0.53
Macro F1 score: 0.48

aaz:
Accuracy score: 0.57
Precision score: 0.59
Recall score: 0.54
Macro F1 score: 0.55

abt:
Accuracy score: 0.54
Precision score: 0.57
Recall score: 0.55
Macro F1 score: 0.53

abx:
Accuracy score: 0.56
Precision score: 0.63
Recall score: 0.55
Macro F1 score: 0.55

aby:
Accuracy score: 0.54
Precision score: 0.53
Recall score: 0.52
Macro F1 score: 0.51

acd:
Accuracy score: 0.47
Precision score: 0.45
Recall score: 0.46
Macro F1 score: 0.42

ace:
Accuracy score: 0.57
Precision score: 0.58
Recall score: 0.53
Macro F1 score: 0.55

acf:
Accuracy score: 0.49
Precision score: 0.47
Recall score: 0.46
Macro F1 score: 0.46

ach:
Accuracy score: 0.55
Precision score: 0.55
Recall score: 0.5
Macro F1

/mounts/data/proj/yihong/newhome/ENTER/envs/concept-net/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


aii:
Accuracy score: 0.52
Precision score: 0.53
Recall score: 0.51
Macro F1 score: 0.49

aim:
Accuracy score: 0.49
Precision score: 0.48
Recall score: 0.49
Macro F1 score: 0.45

aji:
Accuracy score: 0.21
Precision score: 0.23
Recall score: 0.27
Macro F1 score: 0.15

ajz:
Accuracy score: 0.5
Precision score: 0.51
Recall score: 0.5
Macro F1 score: 0.5



/mounts/data/proj/yihong/newhome/ENTER/envs/concept-net/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


akb:
Accuracy score: 0.53
Precision score: 0.54
Recall score: 0.51
Macro F1 score: 0.5

ake:
Accuracy score: 0.5
Precision score: 0.55
Recall score: 0.52
Macro F1 score: 0.5

akh:
Accuracy score: 0.55
Precision score: 0.57
Recall score: 0.55
Macro F1 score: 0.55

ald:
Accuracy score: 0.45
Precision score: 0.36
Recall score: 0.42
Macro F1 score: 0.38

alj:
Accuracy score: 0.48
Precision score: 0.49
Recall score: 0.44
Macro F1 score: 0.43

aln:
Accuracy score: 0.46
Precision score: 0.47
Recall score: 0.51
Macro F1 score: 0.46

alp:
Accuracy score: 0.54
Precision score: 0.52
Recall score: 0.53
Macro F1 score: 0.5

alq:
Accuracy score: 0.48
Precision score: 0.49
Recall score: 0.47
Macro F1 score: 0.47

alt:
Accuracy score: 0.56
Precision score: 0.63
Recall score: 0.53
Macro F1 score: 0.53

alz:
Accuracy score: 0.57
Precision score: 0.62
Recall score: 0.58
Macro F1 score: 0.58

ame:
Accuracy score: 0.52
Precision score: 0.49
Recall score: 0.46
Macro F1 score: 0.46

amf:
Accuracy score: 0.51

...

tpm:
Accuracy score: 0.56
Precision score: 0.48
Recall score: 0.55
Macro F1 score: 0.5

tpp:
Accuracy score: 0.46
Precision score: 0.49
Recall score: 0.48
Macro F1 score: 0.44

tpt:
Accuracy score: 0.5
Precision score: 0.51
Recall score: 0.48
Macro F1 score: 0.47

tpz:
Accuracy score: 0.42
Precision score: 0.41
Recall score: 0.47
Macro F1 score: 0.37

tqb:
Accuracy score: 0.5
Precision score: 0.52
Recall score: 0.49
Macro F1 score: 0.48

trc:
Accuracy score: 0.5
Precision score: 0.44
Recall score: 0.5
Macro F1 score: 0.46

trn:
Accuracy score: 0.55
Precision score: 0.53
Recall score: 0.55
Macro F1 score: 0.53

trq:
Accuracy score: 0.55
Precision score: 0.51
Recall score: 0.55
Macro F1 score: 0.5

tsn:
Accuracy score: 0.6
Precision score: 0.58
Recall score: 0.6
Macro F1 score: 0.58

tsz:
Accuracy score: 0.52
Precision score: 0.47
Recall score: 0.48
Macro F1 score: 0.47

ttc:
Accuracy score: 0.55
Precision score: 0.59
Recall score: 0.51
Macro F1 score: 0.5

tte:
Accuracy score: 0.56
Prec

/mounts/data/proj/yihong/newhome/ENTER/envs/concept-net/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


xsm:
Accuracy score: 0.57
Precision score: 0.49
Recall score: 0.55
Macro F1 score: 0.51

xsu:
Accuracy score: 0.46
Precision score: 0.46
Recall score: 0.43
Macro F1 score: 0.41

xtd:
Accuracy score: 0.5
Precision score: 0.43
Recall score: 0.48
Macro F1 score: 0.45

xtm:
Accuracy score: 0.58
Precision score: 0.6
Recall score: 0.54
Macro F1 score: 0.53

xuo:
Accuracy score: 0.5
Precision score: 0.4
Recall score: 0.44
Macro F1 score: 0.4

yaa:
Accuracy score: 0.55
Precision score: 0.61
Recall score: 0.56
Macro F1 score: 0.56

yad:
Accuracy score: 0.48
Precision score: 0.45
Recall score: 0.49
Macro F1 score: 0.46

yal:
Accuracy score: 0.57
Precision score: 0.51
Recall score: 0.53
Macro F1 score: 0.52

yam:
Accuracy score: 0.48
Precision score: 0.5
Recall score: 0.47
Macro F1 score: 0.44

yan:
Accuracy score: 0.54
Precision score: 0.52
Recall score: 0.52
Macro F1 score: 0.51

yaq:
Accuracy score: 0.52
Precision score: 0.53
Recall score: 0.52
Macro F1 score: 0.52

yby:
Accuracy score: 0.51
P

/mounts/data/proj/yihong/newhome/ENTER/envs/concept-net/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


zpz:
Accuracy score: 0.56
Precision score: 0.57
Recall score: 0.57
Macro F1 score: 0.53

zsm:
Accuracy score: 0.45
Precision score: 0.47
Recall score: 0.43
Macro F1 score: 0.43

zsr:
Accuracy score: 0.55
Precision score: 0.51
Recall score: 0.53
Macro F1 score: 0.51

ztq:
Accuracy score: 0.51
Precision score: 0.48
Recall score: 0.49
Macro F1 score: 0.48

zty:
Accuracy score: 0.52
Precision score: 0.46
Recall score: 0.47
Macro F1 score: 0.46

zul:
Accuracy score: 0.52
Precision score: 0.54
Recall score: 0.52
Macro F1 score: 0.52

zyp:
Accuracy score: 0.6
Precision score: 0.54
Recall score: 0.59
Macro F1 score: 0.54



In [15]:
# computing the average
total_scores = []
for lang, f1 in f1_performance.items():
    total_scores.append(f1)

print(f"Average F1 score on all {len(total_scores)} langauges is {sum(total_scores)/len(total_scores)}")

Average F1 score on all 1245 langauges is 0.49004016064257094


In [16]:
# nt_word

# eng, 860, linear regression: 0.4688835341365472
# arb, 860, linear regression: 0.49004016064257094
# zho, 860, linear regression: 0.48672289156626497

In [17]:
# clique_word

# eng, 860, linear regression: 0.44661316211878127
# arb, 860, linear regression: 0.4400561797752811
# zho, 860, linear regression: 0.4298635634028896

In [18]:
# sentence_id

# eng, 860, linear regression: 0.3155202541699759
# arb, 860, linear regression: 0.28831612390786376
# zho, 860, linear regression: 0.26097696584590985

In [19]:
# emb_name = 'eflomal_node2vec'

# eng, 860, linear regression: 0.47729150119142216
# arb, 860, linear regression: 0.46794281175536157
# zho, 860, linear regression: 0.44868943606036615